In [2]:
import config
import os
os.environ["GOOGLE_API_KEY"] =config.GOOGLE_API_KEY

In [22]:
!pip install -q langchain-google-genai chromadb langchain pypdf langchain_community sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 6.3 MB/s eta 0:00:00


In [23]:
from langchain_google_genai import GoogleGenerativeAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [24]:
loader = PyPDFLoader("/content/drive/MyDrive/Text_book-1-49.pdf")
documents = loader.load()

In [25]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

In [26]:
splits = text_splitter.split_documents(documents)

In [27]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [28]:
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings
)

In [29]:
llm = GoogleGenerativeAI(model="gemini-pro", temperature=0.3)

In [36]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    output_key="answer",
    return_messages=True
)

In [37]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    return_source_documents=True
)

In [38]:
def chat_with_docs(query):
    result = qa_chain({"question": query})
    return result

In [39]:
query = "What is this document about?"
response = chat_with_docs(query)
print(response)

{'question': 'What is this document about?', 'chat_history': [HumanMessage(content='What is this document about?', additional_kwargs={}, response_metadata={}), AIMessage(content='This document is the preface of the book "Machine Learning and Data Science Blueprints for Finance".', additional_kwargs={}, response_metadata={})], 'answer': 'This document is the preface of the book "Machine Learning and Data Science Blueprints for Finance".', 'source_documents': [Document(metadata={'page': 16, 'source': '/content/drive/MyDrive/Text_book-1-49.pdf'}, page_content='We have a web page for this book, where we list errata, examples, and any additional\ninformation. You can access this page at https://oreil.ly/ML-and-data-science-\nblueprints.\nEmail bookquestions@oreilly.com to comment or ask technical questions about this\nbook.\nFor news and information about our books and courses, visit http://oreilly.com.\nFind us on Facebook: http://facebook.com/oreilly\nFollow us on Twitter: http://twitter.